In [1]:
%matplotlib inline

Learn the Basics [html](https://pytorch.org/tutorials/beginner/basics/intro.html), [nb](0_intro.ipynb) ||
Quickstart [html](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html), [nb](1_quickstart_tutorial.ipynb) ||
Tensors [html](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html), [nb](2_tensorqs_tutorial.ipynb) ||
Datasets & DataLoaders [html](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html), [nb](3_data_tutorial.ipynb) ||
Transforms [html](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html), [nb](4_transforms_tutorial.ipynb) ||
**Build Model** [html](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) ||
Autograd [html](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html), [nb](6_autogradqs_tutorial.ipynb) ||
Optimization [html](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html), [nb](7_optimization_tutorial.ipynb) ||
Save & Load Model [html](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html), [nb](8_saveloadrun_tutorial.ipynb)

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The [`torch.nn`](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the [`Fashion-MNIST dataset`](https://github.com/zalandoresearch/fashion-mnist).


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

F:\anaconda3\envs\pyt-gpu\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'F:\anaconda3\envs\pyt-gpu\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
[`torch.cuda`](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we
continue to use the CPU.



In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [`background operations`](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


--------------




Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

The [`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3062,  0.4791,  0.2241,  0.3394, -0.4661, -0.1559,  0.2055,  0.0687,
          0.4755,  0.6506,  0.3664, -0.2714,  0.1323,  0.3477,  0.3364,  0.2308,
          0.4771,  0.0464,  0.0696, -0.2404],
        [ 0.2171,  0.2884,  0.4520,  0.6092, -0.5003, -0.4047,  0.7435,  0.0217,
          0.5797,  0.8366,  0.2382, -0.0891, -0.1520,  0.3662,  0.3393,  0.2410,
          0.3232, -0.1286,  0.0764, -0.2349],
        [ 0.6437,  0.4142,  0.8086,  0.2080, -0.2801, -0.5599,  0.3150, -0.3337,
          0.4730,  0.6530, -0.2767, -0.2358, -0.0362,  0.1920,  0.3065,  0.5253,
          0.4227,  0.2508, -0.0750, -0.1253]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3062, 0.4791, 0.2241, 0.3394, 0.0000, 0.0000, 0.2055, 0.0687, 0.4755,
         0.6506, 0.3664, 0.0000, 0.1323, 0.3477, 0.3364, 0.2308, 0.4771, 0.0464,
         0.0696, 0.0000],
        [0.2171, 0.2884, 0.4520, 0.6092, 0.0000, 0.0000, 0.7435, 0.0217, 0.5797,
         0.8366, 0.2382, 0.0000, 0.0000, 0.3662, 0.33

### nn.Sequential

[`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0124,  0.0097,  0.0070,  ..., -0.0192,  0.0325, -0.0242],
        [ 0.0127, -0.0019,  0.0110,  ..., -0.0080,  0.0265,  0.0334]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0332,  0.0225], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 3.8547e-02, -2.2971e-02, -3.0514e-02,  ..., -3.0740e-02,
         -3.3507e-02,  7.4167e-05],
        [-4.3066e-02, -6.6068e-03,  1.0870e-03,  ...,  3.3367e-02,
      

--------------




Further Reading
--------------
- [`torch.nn API`](https://pytorch.org/docs/stable/nn.html)

